In [27]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import os
import dotenv
from db.database import dbSession
from db.models import CaHousingDatum
from db.crud import write_record
from pprint import pprint
from tqdm import tqdm

In [28]:
dotenv.load_dotenv(".env", override=True)
DATABASE_URI = os.getenv("DATABASE_URI")

In [29]:
db_session = dbSession(DATABASE_URI)
db_session.create_session()

In [30]:
def count_values(write_results):
    # Initialize counters for True, False, and other values.
    true_count = 0
    false_count = 0
    others_count = 0
    
    # Iterate through each value in the list.
    for value in write_results:
        if value is True:
            true_count += 1
        elif value is False:
            false_count += 1
        else:
            others_count += 1
            
    return true_count, false_count, others_count

In [31]:
def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

In [32]:
housing_df = load_housing_data()
housing_df = housing_df.reset_index(names="row_id")

In [33]:
housing_df_records = housing_df.to_dict(orient="records")
len(housing_df_records)

20640

In [34]:
write_results = list()
for rec in housing_df_records:
    r = write_record(db_session.session, CaHousingDatum, rec)
    write_results.append(r)

In [35]:
a, b, c = count_values(write_results)
print(a, b)

0 20640
